# 一、 批量提取深度特征

## 1. 导入包

In [2]:
import os
import nibabel as nib
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
import pandas as pd

2024-08-30 18:27:36.630011: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 18:27:36.634544: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-30 18:27:36.647894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 18:27:36.669239: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 18:27:36.675668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 18:27:36.691905: I tensorflow/core/platform/cpu_feature_gu

In [3]:
# 加载预训练的 ResNet-50 模型，并截取到倒数第二层
base_model = ResNet50(weights='imagenet', include_top=False)
x = GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)

In [24]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, None,      │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ None, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, None,      │      9,472 │ conv1_pad[0][0]   │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, None,      │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, None,      │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, None,      │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, None,      │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, None,      │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, None,      │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, None,      │          0 │ conv2_block1_1_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, None,      │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, None,      │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, None,      │          0 │ conv2_block1_2_b… │
│ (Activation)        │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, None,      │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, None,      │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, None,      │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ None, 256)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, None,      │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [4]:
# 设置图像和掩码的文件夹路径
base_dir = '/home/gg/jupyter_notebook_wd/Radiomice/130_gg'
image_dir = os.path.join(base_dir, 'images/')
mask_dir = os.path.join(base_dir, 'masks/')

In [5]:
# 读取nii.gz文件
def load_nii(file_path):
    return nib.load(file_path).get_fdata()

# 应用窗位窗宽调整
def apply_window(image, window_level, window_width):
    min_value = window_level - window_width / 2
    max_value = window_level + window_width / 2
    windowed_image = np.clip(image, min_value, max_value)
    return (windowed_image - min_value) / (max_value - min_value)

# 将3D图像转换为2D切片并预处理
def get_slices(image, mask, window_level=50, window_width=350):
    slices = []
    for i in range(image.shape[2]):
        slice_2d = image[:, :, i]
        mask_2d = mask[:, :, i]
        if np.max(mask_2d) > 0:  # 仅处理包含病灶的切片
            # 根据掩码提取病灶区域
            coords = np.where(mask_2d > 0)
            min_row, max_row = np.min(coords[0]), np.max(coords[0])
            min_col, max_col = np.min(coords[1]), np.max(coords[1])
            slice_2d = slice_2d[min_row:max_row+1, min_col:max_col+1]
            
            # 应用窗位窗宽调整
            slice_2d = apply_window(slice_2d, window_level, window_width)
            slice_2d[slice_2d == 0] = -1  # 背景归一化

            # 复制到3个通道 (R, G, B)
            slice_rgb = np.stack([slice_2d, slice_2d, slice_2d], axis=-1)

            # z-score 标准化
            mean = np.mean(slice_rgb, axis=(0, 1))
            std = np.std(slice_rgb, axis=(0, 1))
            slice_rgb = (slice_rgb - mean) / std

            # 调整图像大小到 224x224
            slice_resized = img_to_array(array_to_img(slice_rgb).resize((224, 224)))

            slices.append(slice_resized)
    return np.array(slices)


## 2.示例读取一个图像和对应的掩码

In [ ]:

# 示例读取一个图像和对应的掩码
image = load_nii(os.path.join(image_dir, image_files[0]))
mask = load_nii(os.path.join(mask_dir, mask_files[0]))

# 获取病灶区域切片并预处理
lesion_slices = get_slices(image, mask)

features = []

# 对于每个切片单独进行特征提取
for i in range(lesion_slices.shape[0]):
    slice_ = lesion_slices[i]  # 取出单个切片
    slice_ = np.expand_dims(slice_, axis=0)  # 添加批次维度，变为 (1, 224, 224, 3)
    slice_ = preprocess_input(slice_)  # 预处理以匹配模型的输入要求
    feature = model.predict(slice_)  # 提取特征
    features.append(feature)

# 将所有特征堆叠在一起
patient_features = np.mean(np.vstack(features), axis=0)


# 特征数组（features）现在包含了所有切片的特征向量

## 3.批量提取特征

In [6]:
# 列出文件夹中的所有文件
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.nii.gz')])
mask_files = sorted([f for f in os.listdir(mask_dir) if f.endswith('.nii.gz')])

In [10]:
# 初始化存储所有样本特征的列表
all_features = []

# 遍历所有图像和掩码文件
for img_file, mask_file in zip(image_files, mask_files):
    # 读取图像和掩码
    image = load_nii(os.path.join(image_dir, img_file))
    mask = load_nii(os.path.join(mask_dir, mask_file))

    # 获取病灶区域切片并预处理
    lesion_slices = get_slices(image, mask)

    # 初始化存储单个样本特征的列表
    patient_features = []

    # 对于每个切片单独进行特征提取
    for i in range(lesion_slices.shape[0]):
        slice_ = lesion_slices[i]  # 取出单个切片
        slice_ = np.expand_dims(slice_, axis=0)  # 添加批次维度，变为 (1, 224, 224, 3)
        feature = model.predict(slice_)  # 提取特征
        patient_features.append(feature)

    # 聚合特征（例如，使用平均池化）
    patient_features = np.mean(np.vstack(patient_features), axis=0)
    
    # 添加文件名标识符和特征
    all_features.append([img_file, *patient_features])

In [8]:
# 转换为 pandas DataFrame
columns = ['PatientID'] + [f'Feature_{i}' for i in range(len(all_features[0]) - 1)]
df = pd.DataFrame(all_features, columns=columns)

# 保存为CSV文件
df.to_csv('deep_features_with_ids_bz.csv', index=False)

# 二、特征筛选

In [11]:
import pandas as pd
from scipy.stats import ttest_ind, levene
from scipy.stats import mannwhitneyu

In [19]:
dataFile = '/home/gg/jupyter_notebook_wd/Radiomice/浅层、深层特征提取/deep_features_with_ids.csv'
data1 = pd.read_csv(dataFile)
data = data1.iloc[:,1:]
data_a = data[data['survival'] == 0]
data_b = data[data['survival'] == 1]

In [20]:
X_a = data_a.iloc[: ,1:]
y_a = data_a['survival']
X_b = data_b.iloc[: ,1:]
y_b = data_b['survival']
print(X_a.shape, X_b.shape)

(60, 2048) (40, 2048)


## 曼惠特尼方法筛选

In [21]:
#不需要进行方差齐性
colNamesSel_mwU = []
for colName in X_a.columns[:]: 
    if mannwhitneyu(X_a[colName],X_b[colName])[1] < 0.05:
        colNamesSel_mwU.append(colName)
print(len(colNamesSel_mwU))
print(colNamesSel_mwU)

82
['Feature_8', 'Feature_12', 'Feature_18', 'Feature_45', 'Feature_48', 'Feature_53', 'Feature_78', 'Feature_93', 'Feature_165', 'Feature_185', 'Feature_206', 'Feature_219', 'Feature_228', 'Feature_284', 'Feature_286', 'Feature_297', 'Feature_323', 'Feature_332', 'Feature_336', 'Feature_337', 'Feature_373', 'Feature_388', 'Feature_398', 'Feature_455', 'Feature_465', 'Feature_481', 'Feature_483', 'Feature_487', 'Feature_490', 'Feature_516', 'Feature_519', 'Feature_521', 'Feature_546', 'Feature_561', 'Feature_572', 'Feature_579', 'Feature_580', 'Feature_583', 'Feature_645', 'Feature_688', 'Feature_703', 'Feature_717', 'Feature_815', 'Feature_836', 'Feature_854', 'Feature_888', 'Feature_976', 'Feature_995', 'Feature_1012', 'Feature_1053', 'Feature_1121', 'Feature_1130', 'Feature_1135', 'Feature_1175', 'Feature_1227', 'Feature_1305', 'Feature_1341', 'Feature_1396', 'Feature_1412', 'Feature_1416', 'Feature_1424', 'Feature_1426', 'Feature_1436', 'Feature_1447', 'Feature_1495', 'Feature_1500

In [22]:
Out_X = data1[colNamesSel_mwU]
Out_y = data1.iloc[:,:2]

In [23]:
data_combined = pd.concat([Out_y , Out_X ], axis=1)
data_combined.to_csv('deep_feature_output.csv', index=False)